In [35]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
import warnings
warnings.filterwarnings("ignore")


In [37]:
file_train = 'D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_application_train.csv'
df_train = pd.read_csv(file_train,index_col = 0)
file_test = 'D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_application_test.csv'
df_test = pd.read_csv(file_test,index_col = 0)

In [38]:
df_train.head(5)

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_CURR
0,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,278621
1,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,139008
2,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,138348
3,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,...,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,64140
4,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,219374


In [39]:
df_train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246009 entries, 0 to 246008
Data columns (total 122 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    TARGET                        int64  
 1    NAME_CONTRACT_TYPE            object 
 2    CODE_GENDER                   object 
 3    FLAG_OWN_CAR                  object 
 4    FLAG_OWN_REALTY               object 
 5    CNT_CHILDREN                  int64  
 6    AMT_INCOME_TOTAL              float64
 7    AMT_CREDIT                    float64
 8    AMT_ANNUITY                   float64
 9    AMT_GOODS_PRICE               float64
 10   NAME_TYPE_SUITE               object 
 11   NAME_INCOME_TYPE              object 
 12   NAME_EDUCATION_TYPE           object 
 13   NAME_FAMILY_STATUS            object 
 14   NAME_HOUSING_TYPE             object 
 15   REGION_POPULATION_RELATIVE    float64
 16   DAYS_BIRTH                    int64  
 17   DAYS_EMPLOYED                 int64  
 18   DA

In [40]:
# From researching, in order to fill the missing df_train of EXT_SOURCES, I will use 
#Linear Regression(L1 regularlization) to predict the missing df_train of these cols
from sklearn.linear_model import Lasso

def ext_source_predictor():
    '''
    Function to predict the missing df_train of EXT_SOURCE features
    '''
    
    #using only numeric columns for predicting the EXT_SOURCES
    columns_for_modelling = list(set(df_train._get_numeric_data().columns.tolist())
                                            - set(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','SK_ID_CURR','TARGET']))
    
    #we'll train a Linear Regression model for predicting missing EXT_SOURCE
    #we will predict in the order of least number of missing value columns to max.
    for ext_col in ['EXT_SOURCE_2','EXT_SOURCE_3','EXT_SOURCE_1']:
        # Selecting rows with non-NaN values in the specified column for modeling
        X_model = df_train[~df_train[ext_col].isna()][columns_for_modelling]

        # Selecting rows with NaN values in the specified column for training
        X_train_missing = df_train[df_train[ext_col].isna()][columns_for_modelling]

        # Selecting rows with NaN values in the specified column for testing
        X_test_missing = df_test[df_test[ext_col].isna()][columns_for_modelling]

        # Extracting the target variable for training instances
        Y_train = df_train[ext_col][~df_train[ext_col].isna()]

        #model
        LR = Lasso(alpha=1.0)  
        LR.fit(X_model, Y_train)

        df_train[ext_col][df_train[ext_col].isna()] = LR.predict(X_train_missing)
        df_test[ext_col][df_test[ext_col].isna()] = LR.predict(X_test_missing)
        
        #adding the predicted column to columns for modelling for next column's prediction
        columns_for_modelling = columns_for_modelling + [ext_col]

# Features Engineering and Selection

In [41]:
def preprocess(data):
      
        # Housing 
        start = data.columns.get_loc('APARTMENTS_AVG')
        end = data.columns.get_loc('EMERGENCYSTATE_MODE')
        normalize_house_col_drop = list(data.columns[start:end+1])
        data = data.drop(columns = normalize_house_col_drop,axis = 1)

        # day_births and days_employed
        data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].replace(365243,np.nan)


        # fillna by mean for numerical columns except 3 ext cols 
        filled_columns = list(set(data._get_numeric_data().columns.tolist())
                                            - set(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','SK_ID_CURR']))
        for i in filled_columns:
                data[i] = data[i].fillna(data[i].mean())

        return data

In [42]:
def handle_outliers(data):
    pass

In [43]:
df_train = preprocess(df_train)
df_test = preprocess(df_test)
ext_source_predictor()
handle_outliers(df_test)
handle_outliers(df_train)

In [44]:
from sklearn.linear_model import ElasticNet
prev_app = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\Data\\Data\\dseb63_final_project_DP_dataset\\dseb63_previous_application.csv')
prev_app = prev_app[['AMT_CREDIT', 'AMT_ANNUITY', 'CNT_PAYMENT']]
prev_app = prev_app.fillna(prev_app.mean())
prev_app['Ratio'] = prev_app['AMT_CREDIT'] / (prev_app['AMT_ANNUITY'])
prev_app = prev_app.replace([np.inf, -np.inf],np.nan)
prev_app.fillna(prev_app.mean(), inplace= True)

In [45]:
def predict_cntpayment(data, app_data):
    '''input is prev_app and app_data with 4 cols are AMT_CREDIT, AMT_ANNUITY, CNT_PAYMENT, Ratio
        Output is predict series
    '''
    X_train = data[['AMT_CREDIT', 'AMT_ANNUITY','Ratio']]
    Y_train = data['CNT_PAYMENT']
    
    model = ElasticNet(l1_ratio=0.6)
    model.fit(X_train, Y_train)
    return model.predict(app_data)


In [46]:

def add_features(data):
        
        #income and credit features
        data['CREDIT_INCOME_RATIO'] = data['AMT_CREDIT'] / (data['AMT_INCOME_TOTAL'])
        data['CREDIT_ANNUITY_RATIO'] = data['AMT_CREDIT'] / (data['AMT_ANNUITY'])
        data['ANNUITY_INCOME_RATIO'] = data['AMT_ANNUITY'] / (data['AMT_INCOME_TOTAL'])
        data['INCOME_ANNUITY_DIFF'] = data['AMT_INCOME_TOTAL'] - data['AMT_ANNUITY']
        data['CREDIT_GOODS_RATIO'] = data['AMT_CREDIT'] / (data['AMT_GOODS_PRICE'] )
        data['CREDIT_GOODS_DIFF'] = data['AMT_CREDIT'] - data['AMT_GOODS_PRICE']
        data['GOODS_INCOME_RATIO'] = data['AMT_GOODS_PRICE'] / (data['AMT_INCOME_TOTAL'])


        # predict cnt_payment
        data['CNT_PAYMENT_PRED'] = predict_cntpayment(prev_app,data[['AMT_CREDIT', 'AMT_ANNUITY','CREDIT_ANNUITY_RATIO']])
        data['INTEREST_PRED'] = data['CNT_PAYMENT_PRED']*data['AMT_ANNUITY'] - data['AMT_CREDIT']
        data['INTEREST_RATE_PRED'] = 24*data['INTEREST_PRED']/(data['AMT_CREDIT']*(data['CNT_PAYMENT_PRED']+1))
        data['INTEREST_SHARE_PRED'] = data['INTEREST_PRED']/data['AMT_CREDIT']
        #age ratios and diffs
        data['DAYS_BIRTH'] = data['DAYS_BIRTH'].apply(lambda x: int((x * -1) / 365))
        data['DAYS_EMPLOYED'] = data['DAYS_EMPLOYED'].apply(lambda x: int((x * -1) / 365))
        data['AGE_EMPLOYED_DIFF'] = data['DAYS_BIRTH'] - data['DAYS_EMPLOYED']
        data['EMPLOYED_TO_AGE_RATIO'] = data['DAYS_EMPLOYED'] / (data['DAYS_BIRTH'] )

        #car ratios
        data['CAR_EMPLOYED_DIFF'] = data['OWN_CAR_AGE'] - data['DAYS_EMPLOYED']
        data['CAR_EMPLOYED_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_EMPLOYED'])
        data['CAR_AGE_DIFF'] = data['DAYS_BIRTH'] - data['OWN_CAR_AGE']
        data['CAR_AGE_RATIO'] = data['OWN_CAR_AGE'] / (data['DAYS_BIRTH'])

        #family members
        data['CNT_NON_CHILDREN'] = data['CNT_FAM_MEMBERS'] - data['CNT_CHILDREN']
        data['CHILDREN_INCOME_RATIO'] = data['CNT_CHILDREN'] / (data['AMT_INCOME_TOTAL'] )
        data['PER_CAPITA_INCOME'] = data['AMT_INCOME_TOTAL'] / (data['CNT_FAM_MEMBERS'] + 1)

        #flag contact
        data['FLAG_CONTACT'] = data['FLAG_MOBIL'] + data['FLAG_EMP_PHONE']  + data['FLAG_WORK_PHONE']  + data['FLAG_CONT_MOBILE'] 
        + data['FLAG_EMAIL'] + data['FLAG_PHONE']
        data['FLAG_CONTACT'] = data['FLAG_CONTACT'].apply(lambda x: 'Good' if x >= 4 else 'Normal' if 2 <= x < 4 else 'Bad')
        
        #Flag documents
        data['SUM_FLAGS_DOCUMENTS'] = data['FLAG_DOCUMENT_3'] + data['FLAG_DOCUMENT_5'] + data['FLAG_DOCUMENT_6']  + data[
                                'FLAG_DOCUMENT_7'] + data['FLAG_DOCUMENT_8'] + data['FLAG_DOCUMENT_9'] + data[
                                'FLAG_DOCUMENT_11'] + data['FLAG_DOCUMENT_13'] + data['FLAG_DOCUMENT_14'] + data[
                                'FLAG_DOCUMENT_15'] + data['FLAG_DOCUMENT_16'] + data['FLAG_DOCUMENT_17'] + data[
                                'FLAG_DOCUMENT_18'] + data['FLAG_DOCUMENT_19'] + data['FLAG_DOCUMENT_21']
        #flag regions
        data['FLAG_REGIONS'] = data['REG_REGION_NOT_LIVE_REGION'] + data['REG_REGION_NOT_WORK_REGION'] + data['LIVE_REGION_NOT_WORK_REGION']+data[
                                'REG_CITY_NOT_LIVE_CITY'] + data['REG_CITY_NOT_WORK_CITY'] + data['LIVE_CITY_NOT_WORK_CITY']   
        
        #ext_sources
        data['EXT_SOURCE_SUM'] = data['EXT_SOURCE_1'] + data['EXT_SOURCE_2'] + data['EXT_SOURCE_3'] 
        data['EXT_SOURCE_MEAN'] = (data['EXT_SOURCE_1'] + data['EXT_SOURCE_2'] + data['EXT_SOURCE_3'] ) / 3
        data['EXT_SOURCE_MUL'] = data['EXT_SOURCE_1'] * data['EXT_SOURCE_2'] * data['EXT_SOURCE_3'] 
        data['EXT_SOURCE_MAX'] = [max(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
        data['EXT_SOURCE_MIN'] = [min(ele1,ele2,ele3) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
        data['EXT_SOURCE_VAR'] = [np.var([ele1,ele2,ele3]) for ele1, ele2, ele3 in zip(data['EXT_SOURCE_1'], data['EXT_SOURCE_2'], data['EXT_SOURCE_3'])]
        data['WEIGHTED_EXT_SOURCE'] =  data['EXT_SOURCE_1'] *2 + data['EXT_SOURCE_2'] *3 + data['EXT_SOURCE_3'] * 4  

        data['BIRTH_EXT_SOURCES_MUL'] = data['DAYS_BIRTH'] * data['EXT_SOURCE_1']
        data['BIRTH_EXT_SOURCES_DIV'] = data['DAYS_BIRTH'] / data['EXT_SOURCE_1']

        # bureau 
        data['AMT_REQ_CREDIT_BUREAU_TOTAL_DAY'] = (
                data['AMT_REQ_CREDIT_BUREAU_HOUR'] / 24 +
                data['AMT_REQ_CREDIT_BUREAU_DAY'] +
                data['AMT_REQ_CREDIT_BUREAU_MON'] * 30 +
                data['AMT_REQ_CREDIT_BUREAU_YEAR'] * 365 +
                data['AMT_REQ_CREDIT_BUREAU_WEEK'] * 7 +
                data['AMT_REQ_CREDIT_BUREAU_QRT'] * 3 * 30)

In [47]:
add_features(data = df_train)
add_features(data= df_test)

In [48]:
# Handle with outliers


In [49]:
df_train  = df_train.set_index('SK_ID_CURR')
df_test  = df_test.set_index('SK_ID_CURR')

In [50]:
df_train 

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,EXT_SOURCE_SUM,EXT_SOURCE_MEAN,EXT_SOURCE_MUL,EXT_SOURCE_MAX,EXT_SOURCE_MIN,EXT_SOURCE_VAR,WEIGHTED_EXT_SOURCE,BIRTH_EXT_SOURCES_MUL,BIRTH_EXT_SOURCES_DIV,AMT_REQ_CREDIT_BUREAU_TOTAL_DAY
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
278621,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,1.399264,0.466421,0.090209,0.622246,0.311267,0.016118,4.352276,14.007029,144.570272,0.000000
139008,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,1.812593,0.604198,0.218414,0.650442,0.538005,0.002306,5.351638,32.455601,83.313816,726.179990
138348,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,1.541406,0.513802,0.118773,0.666515,0.322738,0.020432,4.509854,35.991828,81.018392,0.000000
64140,0,Cash loans,M,N,Y,0,99000.0,490495.5,27517.5,454500.0,...,1.551028,0.517009,0.126658,0.621226,0.354225,0.013597,4.698734,26.476555,79.919762,455.000000
219374,0,Cash loans,M,Y,Y,0,360000.0,1530000.0,42075.0,1530000.0,...,1.948693,0.649564,0.267915,0.714279,0.540654,0.006001,5.692974,35.381721,73.512535,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90253,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,225000.0,...,1.267209,0.422403,0.043660,0.681632,0.145570,0.048049,4.096061,3.639261,171.738147,726.179990
9027,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,...,1.311191,0.437064,0.041074,0.652492,0.115992,0.053552,3.823787,36.539565,85.824776,726.179990
163598,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,...,1.498607,0.499536,0.087235,0.744026,0.218859,0.046621,3.970654,30.505082,55.105572,395.041667


## 

In [51]:
df_train.to_csv("D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_train.csv")
df_test.to_csv("D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_test.csv")
